In [ ]:
import tensorflow as tf

# Check for GPU
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print("Error enabling GPU memory growth:", e)
else:
    print("No GPU found")


# to test gpu
print("TensorFlow version:", tf.__version__)
print("GPUs available:", tf.config.list_physical_devices("GPU"))

2024-12-06 21:48:32.919874: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-06 21:48:32.939978: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 21:48:32.939992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 21:48:32.940565: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 21:48:32.944247: I tensorflow/core/platform/cpu_feature_guar

GPU memory growth enabled
TensorFlow version: 2.15.0
GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-12-06 21:48:33.962750: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-06 21:48:33.965900: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-06 21:48:33.965920: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [ ]:
import pandas as pd

In [ ]:
# Load all csv
holidays_df = pd.read_csv("holidays_events.csv")
items_df = pd.read_csv("items.csv")
oil_df = pd.read_csv("oil.csv")
stores_df = pd.read_csv("stores.csv")
transactions_df = pd.read_csv("transactions.csv")
train_df = pd.read_csv("filtered_train.csv")

### holidays_df preprocessing

#### NOTE:
* Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer.
* For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12.
* Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
*
Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday

In [ ]:
holidays_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [ ]:
def preprocess_holidays(holidays_df, start_date, end_date):
    """
    Preprocess the holidays DataFrame:
    - Converts the 'date' column to datetime format.
    - Filters rows within the specified date range.
    - Renames the 'description' column to 'holiday_description'.
    - Translates holiday descriptions to English using a dictionary.
    - Drops the original 'holiday_description' column.

    Parameters:
        holidays_df (pd.DataFrame): The holidays DataFrame to preprocess.
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.

    Returns:
        pd.DataFrame: The preprocessed holidays DataFrame.
    """
    # Convert the 'date' column to datetime format
    holidays_df["date"] = pd.to_datetime(holidays_df["date"])

    # Filter the DataFrame for the specified date range
    holidays_df = holidays_df[
        (holidays_df["date"] >= start_date) & (holidays_df["date"] <= end_date)
    ]

    # Rename the 'description' column to 'holiday_description'
    holidays_df = holidays_df.rename(
        columns={
            "description": "holiday_description",
            "locale": "holiday_scope",
            "type": "holiday_type",
            "locale_name": "location",
        }
    )

    # Convert boolean column to integers (1 and 0)
    holidays_df["transferred"] = holidays_df["transferred"].astype(int)

    # Translation dictionary
    translations = {
        "Fundacion de Riobamba": "Foundation of Riobamba",
        "Fundacion de Ambato": "Foundation of Ambato",
        "Fundacion de Ibarra": "Foundation of Ibarra",
        "Cantonizacion de Quevedo": "Cantonization of Quevedo",
        "Dia de Difuntos": "Day of the Dead",
        "Independencia de Cuenca": "Independence of Cuenca",
        "Puente Dia de Difuntos": "Day of the Dead Bridge",
        "Provincializacion Santa Elena": "Provincialization of Santa Elena",
        "Independencia de Guaranda": "Independence of Guaranda",
        "Independencia de Latacunga": "Independence of Latacunga",
        "Black Friday": "Black Friday",
        "Cyber Monday": "Cyber Monday",
        "Fundacion de Quito-1": "Foundation of Quito-1",
        "Fundacion de Quito": "Foundation of Quito",
        "Fundacion de Loja": "Foundation of Loja",
        "Navidad-4": "Christmas-4",
        "Navidad-3": "Christmas-3",
        "Cantonizacion de Salinas": "Cantonization of Salinas",
        "Navidad-2": "Christmas-2",
        "Carnaval": "Carnival",
        "Fundacion de Manta": "Foundation of Manta",
        "Fundacion de Cuenca": "Foundation of Cuenca",
        "Cantonizacion de Riobamba": "Cantonization of Riobamba",
        "Dia del Trabajo": "Labor Day",
        "Cantonizacion del Puyo": "Cantonization of Puyo",
        "Batalla de Pichincha": "Battle of Pichincha",
        "Traslado Batalla de Pichincha": "Relocation of Battle of Pichincha",
        "Cantonizacion de Guaranda": "Cantonization of Guaranda",
        "Fundacion de Guayaquil-1": "Foundation of Guayaquil-1",
        "Fundacion de Guayaquil": "Foundation of Guayaquil",
        "Primer Grito de Independencia": "First Cry of Independence",
        "Traslado Primer Grito de Independencia": "Relocation of First Cry of Independence",
    }

    # Translate the `holiday_description` column using the dictionary
    holidays_df["holiday_description_english"] = holidays_df["holiday_description"].map(
        translations
    )

    # Drop the original 'holiday_description' column
    holidays_df = holidays_df.drop(columns=["holiday_description"])

    # Rename the 'holiday_description_english' column to 'holiday_description'
    holidays_df = holidays_df.rename(
        columns={"holiday_description_english": "holiday_description"}
    )

    return holidays_df


# Preprocess the holidays DataFrame
start_date = "2016-08-15"
end_date = "2017-08-15"
holidays_df = preprocess_holidays(holidays_df, start_date, end_date)
print(holidays_df.head())

          date holiday_type holiday_scope  location  transferred  \
270 2016-08-15      Holiday         Local  Riobamba            0   
271 2016-08-24      Holiday         Local    Ambato            0   
272 2016-09-28      Holiday         Local    Ibarra            0   
273 2016-10-07      Holiday         Local   Quevedo            0   
274 2016-10-09      Holiday      National   Ecuador            0   

          holiday_description  
270    Foundation of Riobamba  
271      Foundation of Ambato  
272      Foundation of Ibarra  
273  Cantonization of Quevedo  
274                       NaN  


### items_df preprocessing

In [ ]:
items_df.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [ ]:
family_value_counts = items_df["family"].unique()
print(family_value_counts)

['GROCERY I' 'CLEANING' 'BREAD/BAKERY' 'DELI' 'POULTRY' 'EGGS'
 'PERSONAL CARE' 'LINGERIE' 'BEVERAGES' 'AUTOMOTIVE' 'DAIRY' 'GROCERY II'
 'MEATS' 'FROZEN FOODS' 'HOME APPLIANCES' 'SEAFOOD' 'PREPARED FOODS'
 'LIQUOR,WINE,BEER' 'BEAUTY' 'HARDWARE' 'LAWN AND GARDEN' 'PRODUCE'
 'HOME AND KITCHEN II' 'HOME AND KITCHEN I' 'MAGAZINES' 'HOME CARE'
 'PET SUPPLIES' 'BABY CARE' 'SCHOOL AND OFFICE SUPPLIES'
 'PLAYERS AND ELECTRONICS' 'CELEBRATION' 'LADIESWEAR' 'BOOKS']


In [ ]:
def preprocess_items(items_df, family_list):
    """
    Preprocess the items DataFrame:
    - Drops the 'class' column.
    - Filters rows based on a list of families.

    Parameters:
        items_df (pd.DataFrame): The items DataFrame to preprocess.
        family_list (list): List of families to include.

    Returns:
        pd.DataFrame: The preprocessed items DataFrame.
    """
    # Drop the 'class' column
    items_df = items_df.drop(columns=["class"], errors="ignore")

    # Filter the DataFrame for the specified families
    filtered_items_df = items_df[items_df["family"].isin(family_list)]

    return filtered_items_df


# List of families to include
family_list = [
    "GROCERY I",
    "BEVERAGES",
    "CLEANING",
    "PRODUCE",
    "DAIRY",
    "BREAD/BAKERY",
    "LIQUOR,WINE,BEER",
    "FROZEN FOODS",
    "PREPARED FOODS",
    "DELI",
]
# Preprocess the items DataFrame
items_df = preprocess_items(items_df, family_list)
print(items_df)

      item_nbr            family  perishable
0        96995         GROCERY I           0
1        99197         GROCERY I           0
2       103501          CLEANING           0
3       103520         GROCERY I           0
4       103665      BREAD/BAKERY           1
...        ...               ...         ...
4095   2132318         GROCERY I           0
4096   2132945         GROCERY I           0
4097   2132957         GROCERY I           0
4098   2134058         BEVERAGES           0
4099   2134244  LIQUOR,WINE,BEER           0

[3320 rows x 3 columns]


### stores_df preprocessing

In [ ]:
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [ ]:
# Rename columns
stores_df = stores_df.rename(
    columns={
        "city": "store_city",
        "state": "store_state",
        "type": "store_type",
        "cluster": "store_cluster",
    }
)
stores_df.head()

,store_nbr,store_city,store_state,store_type,store_cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### oil_df preprocessing

In [ ]:
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [ ]:
# Rename columns
oil_df = oil_df.rename(columns={"dcoilwtico": "oil_price"})
oil_df.head()

,date,oil_price
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


### train_df preprocessing

* Training data, which includes the target unit_sales by date, store_nbr, and item_nbr and a unique id to label rows.
* The target unit_sales can be integer (e.g., a bag of chips) or float (e.g., 1.5 kg of cheese).
* Negative values of unit_sales represent returns of that particular item.
* The onpromotion column tells whether that item_nbr was on promotion for a specified date and store_nbr.
* Approximately 16% of the onpromotion values in this file are NaN.
* NOTE: The training data does not include rows for items that had zero unit_sales for a store/date combination. There is no information as to whether or not the item was in stock for the store on the date, and teams will need to decide the best way to handle that situation. Also, there are a small number of items seen in the training data that aren't seen in the test data.

In [ ]:
train_df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,88042205,2016-08-15,1,103665,1.0,False
1,88042206,2016-08-15,1,105574,1.0,False
2,88042207,2016-08-15,1,105575,19.0,False
3,88042208,2016-08-15,1,105577,1.0,False
4,88042209,2016-08-15,1,105693,1.0,False


In [ ]:
def preprocess_train_df(train_df):
    """
    Preprocess the train DataFrame:
    - Converts the 'date' column to datetime.
    - Extracts temporal features: year, month, month name, and weekday.
    - Converts the 'onpromotion' column from boolean to binary (0 for False, 1 for True).

    Parameters:
        train_df (pd.DataFrame): The train DataFrame to preprocess.

    Returns:
        pd.DataFrame: The preprocessed train DataFrame.
    """
    # Convert 'date' column to datetime format
    train_df["date"] = pd.to_datetime(train_df["date"])

    # # Extract temporal features
    # train_df["year"] = train_df["date"].dt.year
    # train_df["month"] = train_df["date"].dt.month
    # train_df["month_name"] = train_df["date"].dt.strftime("%b")  # Jan, Feb, etc.
    # train_df["weekday"] = train_df["date"].dt.strftime("%a")  # Mon, Tue, etc.

    # Convert 'onpromotion' column from boolean to binary
    train_df["onpromotion"] = train_df["onpromotion"].astype(
        int
    )  # 1 for True, 0 for False

    return train_df


# Preprocess the train DataFrame
train_df = preprocess_train_df(train_df)
train_df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,88042205,2016-08-15,1,103665,1.0,0
1,88042206,2016-08-15,1,105574,1.0,0
2,88042207,2016-08-15,1,105575,19.0,0
3,88042208,2016-08-15,1,105577,1.0,0
4,88042209,2016-08-15,1,105693,1.0,0


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37454835 entries, 0 to 37454834
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   item_nbr     int64         
 4   unit_sales   float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 1.7 GB


In [ ]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [ ]:
holidays_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57 entries, 270 to 326
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 57 non-null     datetime64[ns]
 1   holiday_type         57 non-null     object        
 2   holiday_scope        57 non-null     object        
 3   location             57 non-null     object        
 4   transferred          57 non-null     int64         
 5   holiday_description  35 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 3.1+ KB


In [ ]:
oil_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       1218 non-null   object 
 1   oil_price  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [ ]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3320 entries, 0 to 4099
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   item_nbr    3320 non-null   int64 
 1   family      3320 non-null   object
 2   perishable  3320 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 103.8+ KB


In [ ]:
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   store_nbr      54 non-null     int64 
 1   store_city     54 non-null     object
 2   store_state    54 non-null     object
 3   store_type     54 non-null     object
 4   store_cluster  54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


### Merge all df into 1 df

In [ ]:
# Ensure date column in transactions_df is datetime
transactions_df['date'] = pd.to_datetime(transactions_df['date'])

# Ensure date column in oil_df is datetime
oil_df['date'] = pd.to_datetime(oil_df['date'])

chunk_size = 1000  # Number of rows per chunk
n_chunks = len(train_df) // chunk_size + 1  # Calculate the number of chunks
chunks = []

# Process train_df in chunks
for i in range(n_chunks):
    # Slice the chunk
    chunk = train_df.iloc[i * chunk_size:(i + 1) * chunk_size]

    # Perform merges
    chunk = pd.merge(chunk, transactions_df, on=['date', 'store_nbr'], how='left')
    chunk = pd.merge(chunk, holidays_df, on='date', how='left')
    chunk = pd.merge(chunk, oil_df, on='date', how='left')
    chunk = pd.merge(chunk, items_df, on='item_nbr', how='left')
    chunk = pd.merge(chunk, stores_df, on='store_nbr', how='left')

    # Append the processed chunk to the list
    chunks.append(chunk)

# Concatenate all processed chunks
merged_df = pd.concat(chunks, ignore_index=True)

# Verify the final DataFrame
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38100308 entries, 0 to 38100307
Data columns (total 19 columns):
 #   Column               Dtype         
---  ------               -----         
 0   id                   int64         
 1   date                 datetime64[ns]
 2   store_nbr            int64         
 3   item_nbr             int64         
 4   unit_sales           float64       
 5   onpromotion          int64         
 6   transactions         float64       
 7   holiday_type         object        
 8   holiday_scope        object        
 9   location             object        
 10  transferred          float64       
 11  holiday_description  object        
 12  oil_price            float64       
 13  family               object        
 14  perishable           float64       
 15  store_city           object        
 16  store_state          object        
 17  store_type           object        
 18  store_cluster        int64         
dtypes: datetime64[ns](1

In [ ]:
# dropping missing values from merged_df and creating filtered_df
filtered_df = merged_df.dropna(subset=[
    'holiday_type',
    'holiday_scope',
    'location',
    'transferred',
    'holiday_description',
    'oil_price',
    'transactions'
])

filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3563105 entries, 0 to 38100307
Data columns (total 19 columns):
 #   Column               Dtype         
---  ------               -----         
 0   id                   int64         
 1   date                 datetime64[ns]
 2   store_nbr            int64         
 3   item_nbr             int64         
 4   unit_sales           float64       
 5   onpromotion          int64         
 6   transactions         float64       
 7   holiday_type         object        
 8   holiday_scope        object        
 9   location             object        
 10  transferred          float64       
 11  holiday_description  object        
 12  oil_price            float64       
 13  family               object        
 14  perishable           float64       
 15  store_city           object        
 16  store_state          object        
 17  store_type           object        
 18  store_cluster        int64         
dtypes: datetime64[ns](1), flo

### Sampling 1 lac record from merged_df
* Stratified Sampling by a Key Feature (e.g., family)
* project emphasizes understanding the impact of holidays, promotions, and economic indicators across different product categories, it’s important that sample reflects the variety of items sold. A stratified sampling approach by the family column will ensure that each product category is proportionally represented in the sample, preserving important distributions for downstream modeling and analysis.
* Stratified sampling ensures that the complexity and variety of the dataset’s product categories are maintained in 100,000-record subset. This gives a more representative mini-dataset for exploring trends, running initial model prototypes, and understanding the effect of promotions and holidays without losing crucial diversity in data.

In [ ]:
# Calculate the overall distribution of each family
family_distribution = filtered_df['family'].value_counts(normalize=True)

# Determine how many records to sample from each family category
samples_per_family = (family_distribution * 100000).astype(int)

# Stratified sample DataFrame
sample_df = pd.DataFrame()

# Loop through each family and sample proportionally
for fam, count in samples_per_family.items():
    # Filter the dataset by the current family
    fam_subset = filtered_df[filtered_df['family'] == fam]
    # Randomly sample the required number of records
    fam_sample = fam_subset.sample(n=count, random_state=42)
    # Concatenate to the growing sample_df
    sample_df = pd.concat([sample_df, fam_sample], ignore_index=True)

In [ ]:
# changine type from float to int as these are boolen feauture
sample_df['transferred'] = sample_df['transferred'].astype('int64')
sample_df['perishable'] = sample_df['perishable'].astype('int64')

In [ ]:
print(sample_df.describe())

                 id                           date     store_nbr  \
count  9.999500e+04                          99995  99995.000000   
mean   1.060751e+08  2017-02-10 08:31:14.397719808     27.919436   
min    8.804220e+07            2016-08-15 00:00:00      1.000000   
25%    9.657521e+07            2016-11-11 00:00:00     13.000000   
50%    1.008206e+08            2016-12-22 00:00:00     28.000000   
75%    1.153901e+08            2017-05-12 00:00:00     43.000000   
max    1.254970e+08            2017-08-15 00:00:00     54.000000   
std    1.103949e+07                            NaN     16.152617   

           item_nbr    unit_sales   onpromotion  transactions   transferred  \
count  9.999500e+04  99995.000000  99995.000000  99995.000000  99995.000000   
mean   1.128492e+06      8.523352      0.138677   1939.276414      0.084494   
min    9.699500e+04     -2.000000      0.000000    383.000000      0.000000   
25%    6.554270e+05      2.000000      0.000000   1139.000000      0.00

In [ ]:
negative_count = (sample_df['unit_sales'] < 0).sum()
print("Number of negative unit_sales:", negative_count)

Number of negative unit_sales: 8


In [ ]:
# creating a flag is_return indicating return on specific record
sample_df['is_return'] = (sample_df['unit_sales'] < 0).astype(int)
# sample_df.loc[sample_df['unit_sales'] < 0, 'unit_sales'] = abs(sample_df['unit_sales'])

In [ ]:
sample_df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,holiday_type,holiday_scope,location,transferred,holiday_description,oil_price,family,perishable,store_city,store_state,store_type,store_cluster,is_return
0,119882896,2017-06-23,44,939663,2.0,0,4465.0,Holiday,Local,Guaranda,0,Cantonization of Guaranda,42.86,GROCERY I,0,Quito,Pichincha,A,5,0
1,113178951,2017-04-21,43,2026893,2.0,0,1373.0,Holiday,Local,Riobamba,0,Cantonization of Riobamba,49.64,GROCERY I,0,Esmeraldas,Esmeraldas,E,10,0
2,125043190,2017-08-11,38,2010233,1.0,0,1577.0,Transfer,National,Ecuador,0,Relocation of First Cry of Independence,48.81,GROCERY I,0,Loja,Loja,D,4,0
3,88996056,2016-08-24,53,1975578,2.0,0,1259.0,Holiday,Local,Ambato,0,Foundation of Ambato,46.29,GROCERY I,0,Manta,Manabi,D,13,0
4,98298830,2016-11-28,45,1160754,10.0,0,3356.0,Event,National,Ecuador,0,Cyber Monday,45.66,GROCERY I,0,Quito,Pichincha,A,11,0


In [ ]:
# Count missing values in each column
missing_values = sample_df.isnull().sum()

# Display missing values
print(missing_values)

id                     0
date                   0
store_nbr              0
item_nbr               0
unit_sales             0
onpromotion            0
transactions           0
holiday_type           0
holiday_scope          0
location               0
transferred            0
holiday_description    0
oil_price              0
family                 0
perishable             0
store_city             0
store_state            0
store_type             0
store_cluster          0
is_return              0
dtype: int64


In [ ]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99995 entries, 0 to 99994
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   99995 non-null  int64         
 1   date                 99995 non-null  datetime64[ns]
 2   store_nbr            99995 non-null  int64         
 3   item_nbr             99995 non-null  int64         
 4   unit_sales           99995 non-null  float64       
 5   onpromotion          99995 non-null  int64         
 6   transactions         99995 non-null  float64       
 7   holiday_type         99995 non-null  object        
 8   holiday_scope        99995 non-null  object        
 9   location             99995 non-null  object        
 10  transferred          99995 non-null  int64         
 11  holiday_description  99995 non-null  object        
 12  oil_price            99995 non-null  float64       
 13  family               99995 non-

In [ ]:
# Save DataFrame to a CSV file
sample_df.to_csv('preprocessed_data.csv', index=False)

# Verify the file
print("DataFrame has been saved to 'preprocessed_data.csv'.")

DataFrame has been saved to 'preprocessed_data.csv'.
